In [155]:
import pytesseract
import cv2
from PIL import Image
import pandas as pd
import numpy as np

In [156]:
path = 'C:\\Users\\kakut\\OneDrive\\Desktop\\ocr images\\'
image_file = f'{path}photo_2024-02-21_01-53-51.jpg'
img = cv2.imread(image_file)

In [157]:
ocr_result  = pytesseract.image_to_string(img)
print(ocr_result)

Random Wendy’s Old Fashioned Salad Random Wendy's Old Fashioned Salad

number Hamburgers store bar number Hamburgers store bar
00 ~—-5555 Airport Hwy “S-—09~—s« 3124 Monroe St s
01 6525 Airport Hwy s 10 27393 Helen Dr NS
02 5166 Airport Hwy At Reynolds Rd NS 11 4277 Monroe St iS)
03 E Alexis Rd & Telegraph Rd Ss 12 5804 Monroe S
04 2124 W Alexis Rd Ss 13 2866 Navarre Av S
05 5560 W. Central Av Ss 14 3435 Secor Rd NS
06 914 Conant St NS 15 1109 South Av NS
07 3454 Dorr St Ss 16 3465 Stickney Av S
08 Front St & Main St S 17 1945 Woodville Rd Ss



In [158]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imwrite(f"{path}\\temp\\index_gray.png", gray)


True

In [159]:
_, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
cv2.imwrite(f"{path}\\temp\\index_thresh.png", thresh)


True

In [160]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
dilated = cv2.dilate(thresh, kernel, iterations=5)
cv2.imwrite(f"{path}\\temp\\index_dilated.png", dilated)


True

In [161]:
eroded = cv2.erode(dilated, kernel, iterations=1)
cv2.imwrite(f"{path}\\temp\\index_eroded.png", eroded)

True

In [162]:
cnts = cv2.findContours(eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])


In [163]:
cnts

[array([[[  4,  14]],
 
        [[  3,  15]],
 
        [[  0,  15]],
 
        [[  0,  82]],
 
        [[ 10,  82]],
 
        [[ 11,  83]],
 
        [[ 67,  83]],
 
        [[ 68,  84]],
 
        [[ 68,  90]],
 
        [[ 67,  91]],
 
        [[ 34,  91]],
 
        [[ 30,  95]],
 
        [[ 30,  97]],
 
        [[ 29,  98]],
 
        [[ 29, 119]],
 
        [[ 30, 120]],
 
        [[ 30, 127]],
 
        [[ 29, 128]],
 
        [[ 29, 129]],
 
        [[ 28, 130]],
 
        [[ 28, 162]],
 
        [[ 27, 163]],
 
        [[ 27, 165]],
 
        [[ 26, 166]],
 
        [[ 26, 190]],
 
        [[ 27, 191]],
 
        [[ 27, 196]],
 
        [[ 25, 198]],
 
        [[ 25, 201]],
 
        [[ 24, 202]],
 
        [[ 24, 223]],
 
        [[ 25, 224]],
 
        [[ 25, 230]],
 
        [[ 24, 231]],
 
        [[ 24, 233]],
 
        [[ 23, 234]],
 
        [[ 23, 260]],
 
        [[ 24, 261]],
 
        [[ 24, 265]],
 
        [[ 22, 267]],
 
        [[ 22, 271]],
 
        [[ 21, 2

In [164]:
results  = []
rois = []
for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    if h > 200 and w > 20:
        roi = img[y:y+h, x:x+w]
        rois.append(roi)
        cv2.rectangle(img, (x, y), (x+w, y+h), (36, 255, 12), 2)
        ocr_result = pytesseract.image_to_string(roi)
        ocr_result = ocr_result.split("\n")
        for item in ocr_result:
            results.append(item)
cv2.imwrite(f"{path}\\temp\\index_bbox_new.png", img)

True

Extracting the first columns

In [195]:
# Extract text from the first and fifth ROIs
roi1_text = pytesseract.image_to_string(rois[1])
roi4_text = pytesseract.image_to_string(rois[4])

# Strings to exclude from the extracted text
exclude_strings = ("", "Random", "number", "Wendy's Old Fé", "Hamburgers","Wendy's Old Fashioned","Hamburgers store")

# Combine text from both ROIs and filter out specified strings
col1 = [i for i in roi1_text.split("\n") + roi4_text.split("\n") if i.strip() and i not in exclude_strings]

# Get the length of the combined list
len(col1)

18

Column2

In [199]:
col2 = ["S","S","NS","S","S","S","NS","S","S","S","NS","S","S","S","NS","NS","S","S",]

Creating a DataFrame

In [204]:
df = pd.DataFrame({'Store':col1,'Salad bar':col2})
df

,Store,Salad bar
0,5555 Airport Hwy,S
1,6525 Airport Hwy,S
2,5166 Airport Hwy At Reynolds Rd,NS
3,E Alexis Rd & Telegraph Rd,S
4,2124 W Alexis Rd,S
5,5560 W. Central Av,S
6,914 Conant St,NS
7,3454 Dorr St,S
8,Front St & Main St,S
9,3124 Monroe St,S


In [206]:
random_samples = df.sample(n=5)
print(random_samples)

                        Store Salad bar
1            6525 Airport Hwy         S
3  E Alexis Rd & Telegraph Rd         S
4            2124 W Alexis Rd         S
6               914 Conant St        NS
9              3124 Monroe St         S


In [207]:
start_row = 2
step = 5
systematic_sample = df.iloc[start_row::step]
systematic_sample

,Store,Salad bar
2,5166 Airport Hwy At Reynolds Rd,NS
7,3454 Dorr St,S
12,5804 Monroe,S
17,1945 Woodville Rd,S


In [208]:
sample_s = df.loc[df['Salad bar']=='S'].sample(n=3)
sample_ns = df.loc[df['Salad bar']=='NS'].sample(n=1)

sample = pd.concat([sample_s,sample_ns])
sample

,Store,Salad bar
13,2866 Navarre Av,S
12,5804 Monroe,S
11,4277 Monroe St,S
10,27393 Helen Dr,NS
